<a href="https://www.kaggle.com/code/tarekyahia/feature-extraction-approach?scriptVersionId=160123918" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

Besmy allah

# functions


In [1]:
import pandas as pd
import numpy as np
import os,re,time,math,gc,random
import scipy as sp
import torch
import torch.nn.functional as F
from torch import nn 
from tqdm.auto import tqdm
from datasets import Dataset,DatasetDict,Value
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam, SGD, AdamW
from datasets import DatasetDict
from transformers import AutoModelForSequenceClassification,AutoTokenizer
from transformers import TrainingArguments,Trainer
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedGroupKFold,GroupKFold,KFold
from copy import deepcopy
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
os.environ["TOKENIZERS_PARALLELISM"] = "false"
#pd.set_option('max_colwidth', 400)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# utile

if not os.path.exists('models'):
    os.makedirs('models')
OUTPUT_DIR = '/kaggle/working/models/'


In [3]:
class CFG:
    tokenizer = None
    model_path = '/kaggle/input/debertav3base'
    debug=False    
    model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=1)
    n_fold = 5
    seed = 42
    lr = 8e-5
    weight_decay = 0.01
    batch_size = 16
    epochs = 5
    compute_loss = None
    fc_dropout=0.2
    target_size=1
    max_len=512 # use when infer only, otherwise we will use the batch max len
    num_workers=4
    scheduler='cosine' # ['linear', 'cosine']
    batch_scheduler=True
    num_cycles=0.5
    num_warmup_steps=0
    encoder_lr= 2e-5
    decoder_lr= 1e-3
    num_workers= 4
    eps=1e-6
    betas=(0.9, 0.999)
    gradient_accumulation_steps=4
    max_grad_norm=1000
    apex = True
    print_freq=100
    awp = True
    awp_start_epoch = 2
    last_batch_max_len = 512
    
    
    
if CFG.debug == True:
    CFG.epochs = 2
    CFG.tr_n_fold = 1

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/debertav3base and are newly initialized: ['classifier.weight', 'classifier.bias', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# ====================================================
# tokenizer
# ====================================================
tokenizer = AutoTokenizer.from_pretrained(CFG.model_path)
CFG.tokenizer = tokenizer

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


### dataset functions

In [5]:
# batch max_len to overcome the GPU limitations problem
def tokenize_text(text):
    '''
    tokenize to sorting purpose
    '''
    inputs = CFG.tokenizer(text,
                           add_special_tokens=True,
                #           max_length=CFG.max_len,
                 #          padding="max_length",
                          # return_token_type_ids = False,
                          # return_attention_mask = False,
                           return_offsets_mapping=False) # pass only text
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs   


def sort_df_batch_lens(df,bs):
    '''
    sort df by the inputs tokenize count then add specific max len for every batch, instead of the max_len for all, this is more and more efficient to the GPU ram 
    '''
    
    dst_df = deepcopy(df)

    # sort by token num
    length = [len(tokenize_text(inputs)['input_ids']) for inputs in dst_df['inputs']]
    
    #   length = len(tokenizer(text, context_text, add_special_tokens=True)['input_ids'])
    #   input_lengths.append(length)   
    dst_df['input_lengths'] = length
    #print(dst_df['input_lengths'])
    length_sorted_idx = np.argsort([-l for l in length])
    
    # sort dataframe
    sort_df = dst_df.iloc[length_sorted_idx].copy()
    
    
    #print(dst_df.iloc[length_sorted_idx].head())
    
    # calc max_len per batch
    sorted_input_length = sort_df['input_lengths'].values
    batch_max_length = np.zeros_like(sorted_input_length)
    for i in range((len(sorted_input_length)//bs)+1):
        if len(sorted_input_length[i*bs:(i+1)*bs]) != 0: # solve issue, in some batches the array is empty
            CFG.last_batch_max_len = np.max(sorted_input_length[i*bs:(i+1)*bs])
        batch_max_length[i*bs:(i+1)*bs] = CFG.last_batch_max_len # last batch if the array is empty
    sort_df['batch_max_length'] = batch_max_length
    
    return sort_df

In [6]:
# df -> Dataset
def prepare_text(cfg,text,batch_len = CFG.max_len):
    inputs = cfg.tokenizer(text,
                           add_special_tokens=True,
                           max_length=batch_len,
                           padding="max_length",
                          # return_token_type_ids = False,
                          # return_attention_mask = False,
                           return_offsets_mapping=False) # pass only text
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs   

class TrainDataset(torch.utils.data.Dataset):
    def __init__(self, cfg, df):
        # super(TrainDataset, self).__init__(self)
        self.cfg = cfg
        self.batch_max_len = df['batch_max_length'].values
        # self.texts = df['inputs'].values
        self.anchor = df['anchor'].values
        self.targets_list = df['targets_list'].values
        self.context_text = df['context_text'].values
        self.target = df['target'].values
        self.sep = '[SEP]'
        self.labels = df['score'].values

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, item):
        # for shuffling the data after every batch
        # the targets is shared with very little chancge for every group, shuffle is a trick 
        # train['anchor'] + '[SEP]' + train['target'] + '[SEP]'  + train['context_text'] + '[SEP]' + train['targets']
        random.shuffle(self.targets_list[item]) # shuffle in-place every call
        self.targets = '; '.join(self.targets_list[item]) # from list to one long string
        self.input = self.anchor[item] + self.sep + self.target[item] + self.sep  + self.context_text[item] + self.sep + self.targets # the item input
       # inputs['input_ids'] = prepare_text(self.cfg, self.texts[item])['input_ids']
        inputs = prepare_text(self.cfg, self.input,self.batch_max_len[item])
        inputs['label'] = torch.tensor(self.labels[item], dtype=torch.float)
        return inputs

In [7]:
def head(n,dataset):
    """
    like head in df but work with dataset object
    """
    # Use Python slicing to get the first n samples
    for i in range(min(n, len(dataset))):
        sample = dataset[i]
        print(f"Sample {i + 1}: {sample}")

### evaluation mertic 

In [8]:
def get_score(y_true, y_pred):
    score = sp.stats.pearsonr(y_true, y_pred)[0]
    return score

### custom loss function ( pearson corellation )

In [9]:
def pearson(x, y, smooth = 0):

    mean_x = torch.mean(x)
    mean_y = torch.mean(y)
    xm = x.sub(mean_x)
    ym = y.sub(mean_y)  
    
    # Explicitly cast to Half data type
    xm = xm.to(dtype=torch.float16)
    ym = ym.to(dtype=torch.float16)  

    r_num = xm.dot(ym)
    r_den = torch.norm(xm, 2) * torch.norm(ym, 2)
    r_den = torch.clamp(r_den, min=1e-6)
    r_val = (r_num + smooth) / (r_den + smooth) 
    r_val = torch.clamp(r_val, min=-1., max=1.)
    return r_val
      
# PyTorch
class pearsonLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(pearsonLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        # sigmoid work well 
        inputs = F.sigmoid(inputs) # test 
        
        # pearson correlation
        val = pearson(inputs.view(-1), targets.view(-1))
        
        # clamp : to insure values in the range
        # turn the value to loss, the trick : shift the range from -1 to 1, to from 0 to 2, and then turn to loss in the direction of 1.
        loss = 2. - (torch.clamp(val, min=-1., max=1.) + 1.)
        
        return loss
    

In [10]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

## Import and EDA

In [11]:
path = '../input/us-patent-phrase-to-phrase-matching'

Let's set a path to our data:

### into datafram

In [12]:
train = pd.read_csv(path+'/train.csv')
#train = train.sample(200).reset_index()
test = pd.read_csv(path+'/test.csv')

In [13]:
train

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [14]:
print(train.shape)
print(test.shape)

(36473, 5)
(36, 4)


## validation

In [15]:
# ====================================================
# CV split
# ====================================================
train['score_map'] = train['score'].map({0.00: 0, 0.25: 1, 0.50: 2, 0.75: 3, 1.00: 4}) # for the Stratified ( convert to int )
Fold = StratifiedGroupKFold(n_splits=CFG.n_fold, shuffle=True, random_state=CFG.seed)
for n, (train_index, val_index) in enumerate(Fold.split(train, train['score_map'],train['anchor'])):
    train.loc[val_index, 'fold'] = int(n)
train['fold'] = train['fold'].astype(int)
display(train.groupby('fold').size())

fold
0    7334
1    6945
2    7271
3    7399
4    7524
dtype: int64

## more related data.

In [16]:
# ====================================================
# CPC Data
# ====================================================
def get_cpc_texts():
    '''
    return : Dictionary
    '''
    contexts = []
    pattern = '[A-Z]\d+'
    for file_name in os.listdir('../input/cpc-data/CPCSchemeXML202105'):
        result = re.findall(pattern, file_name)
        if result:
            contexts.append(result)
    contexts = sorted(set(sum(contexts, [])))
    results = {}
    for cpc in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'Y']:
        with open(f'../input/cpc-data/CPCTitleList202202/cpc-section-{cpc}_20220201.txt') as f:
            s = f.read()
        pattern = f'{cpc}\t\t.+'
        result = re.findall(pattern, s)
        cpc_result = result[0].lstrip(pattern)
        for context in [c for c in contexts if c[0] == cpc]:
            pattern = f'{context}\t\t.+'
            result = re.findall(pattern, s)
            results[context] = cpc_result + ". " + result[0].lstrip(pattern)
    return results

In [17]:
cpc_texts = get_cpc_texts()
# torch.save(cpc_texts, OUTPUT_DIR+"cpc_texts.pth")
train['context_text'] = train['context'].map(cpc_texts)
test['context_text'] = test['context'].map(cpc_texts)
#display(train.head())
# display(test.head())

## more tricks in the data

In [18]:
targets = train.groupby(['anchor', 'context'])['target'].apply(list).reset_index(name='targets')
train = pd.merge(train, targets, on=['anchor','context'], how='inner') # join
train['targets_list'] = train.apply(lambda row: [item for item in row['targets'] if item != row['target']], axis=1) # delete 
train['n_targets'] = train.targets.apply(len) # for sorting if needed
train['targets'] = train.targets_list.apply(lambda l: '; '.join(l)) # add ; from list to one row
# new input
train['inputs'] = train['anchor'] + '[SEP]' + train['target'] + '[SEP]'  + train['context_text'] + '[SEP]' + train['targets'] # to pass to 'sort_df_batch_lens' and transforemer approach
train.head(2)

,id,anchor,target,context,score,score_map,fold,context_text,targets,targets_list,n_targets,inputs
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,2,0,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,act of abating; active catalyst; eliminating p...,"[act of abating, active catalyst, eliminating ...",21,abatement[SEP]abatement of pollution[SEP]HUMAN...
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,3,0,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement of pollution; active catalyst; elimi...,"[abatement of pollution, active catalyst, elim...",21,abatement[SEP]act of abating[SEP]HUMAN NECESSI...


In [19]:
train['inputs'][0]

'abatement[SEP]abatement of pollution[SEP]HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; COFFEE MILLS; SPICE MILLS; SUCTION CLEANERS IN GENERAL[SEP]act of abating; active catalyst; eliminating process; forest region; greenhouse gases; increased rate; measurement level; minimising sounds; mixing core materials; multi pollution abatement device; noise reduction; pollution abatement; pollution abatement incinerator; pollution certificate; rent abatement; sorbent material; source items pollution abatement technology; stone abutments; tax abatement; water bodies'

In [20]:
# do the same to the test data

targets = test.groupby(['anchor', 'context'])['target'].apply(list).reset_index(name='targets')
test = pd.merge(test, targets, on=['anchor','context'], how='inner') # join
test['targets_list'] = test.apply(lambda row: [item for item in row['targets'] if item != row['target']], axis=1) # delete 
test['n_targets'] = test.targets.apply(len) # not important
test['targets'] = test.targets_list.apply(lambda l: '; '.join(l)) # add ; from list to one row
# new input
#test['inputs'] = test['anchor'] + '[SEP]' + test['target'] + '[SEP]'  + test['context_text'] + '[SEP]' + test['targets'] # to pass to 'sort_df_batch_lens'
test.head(2)

,id,anchor,target,context,context_text,targets,targets_list,n_targets
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,PHYSICS. OPTICS,,[],1
1,09e418c93a776564,adjust gas flow,altering gas flow,F23,MECHANICAL ENGINEERING; LIGHTING; HEATING; WEA...,,[],1


### feature extraction approach :
  - to do this, built the nn with pytorch
      - take the last hidden layer output as input to the new layears
      - build the nn
      - train it
      - see result
      - test different loss functions
      - test different initialization  

### pytorch Functions

In [21]:
# ==========================
# model (pytorch)
# ==========================

class CustomModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        
        # the pretrained model
        self.config = AutoConfig.from_pretrained(cfg.model_path, output_hidden_states=True)
        self.model = AutoModel.from_pretrained(cfg.model_path, config=self.config) # model without additional layers fro any task
        
        # layers
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, self.cfg.target_size)
        self._init_weights(self.fc)
        self.attention = nn.Sequential( # operation like attention, try to know the important parts 
            nn.Linear(self.config.hidden_size, 512),
            nn.Tanh(),
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )
        self._init_weights(self.attention)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs) 
        last_hidden_states = outputs[0] 
        # feature = torch.mean(last_hidden_states, 1)
        weights = self.attention(last_hidden_states) # apply attention 
        feature = torch.sum(weights * last_hidden_states, dim=1) # Compute Weighted Sum for Feature Representation:
        return feature
        
    def forward(self,inputs):
        feature = self.feature(inputs) # apply feature
        output = self.fc(self.fc_dropout(feature)) # 
        return output        

In [22]:
# helper function

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count



def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))

In [23]:
def train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device,awp):
    '''
    one epoch
    '''
    model.train() # train mode
    scaler = torch.cuda.amp.GradScaler(enabled=CFG.apex) # scale the gradents 
    losses = AverageMeter()
    start = end = time.time()
    global_step = 0
    for step, inputs in enumerate(train_loader):
        
        labels = inputs.pop('label').to(device)
        inputs['input_ids'] = inputs['input_ids'].to(device)#[:, :CFG.f_max_len]
        inputs['attention_mask'] = inputs['attention_mask'].to(device)#[:, :CFG.f_max_len]
        inputs['token_type_ids'] = inputs['token_type_ids'].to(device)#[:, :CFG.f_max_len]
        #inputs.pop('attention_mask')
        #inputs.pop('token_type_ids')
        batch_size = labels.size(0)
        
        if  awp is not None and epoch >= CFG.awp_start_epoch:
            awp.perturb(inputs['input_ids'], inputs['attention_mask'], inputs['token_type_ids'], labels, criterion)
                
        with torch.cuda.amp.autocast(enabled=CFG.apex):
            y_preds = model(inputs)
        loss = criterion(y_preds.view(-1, 1), labels.view(-1, 1))
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        
        scaler.scale(loss).backward()
        
        if  awp is not None and epoch >= CFG.awp_start_epoch:
            awp.restore()
            
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            global_step += 1
            
            if CFG.batch_scheduler:
                scheduler.step()
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  'LR: {lr:.8f}  '
                  .format(epoch+1, step, len(train_loader), 
                          remain=timeSince(start, float(step+1)/len(train_loader)),
                          loss=losses,
                          grad_norm=grad_norm,
                          lr=scheduler.get_lr()[0]))

    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    losses = AverageMeter()
    model.eval()
    preds = []
    start = end = time.time()
    for step, inputs, in enumerate(valid_loader):
        
        labels = inputs.pop('label').to(device)
        inputs['input_ids'] = inputs['input_ids'].to(device)#[:, :CFG.f_max_len]
        inputs['attention_mask'] = inputs['attention_mask'].to(device)#[:, :CFG.f_max_len]
        inputs['token_type_ids'] = inputs['token_type_ids'].to(device)#[:, :CFG.f_max_len]
        #inputs.pop('attention_mask')
        #inputs.pop('token_type_ids')
        batch_size = labels.size(0)
        batch_size = labels.size(0)
        
        with torch.no_grad():
            y_preds = model(inputs)
        loss = criterion(y_preds.view(-1, 1), labels.view(-1, 1))
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        preds.append(y_preds.sigmoid().to('cpu').numpy())
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(step, len(valid_loader),
                          loss=losses,
                          remain=timeSince(start, float(step+1)/len(valid_loader))))
    predictions = np.concatenate(preds)
    predictions = np.concatenate(predictions)
    return losses.avg, predictions


In [24]:
def train_loop(folds, fold):

    # ====================================================
    # loader
    # ====================================================
    # folds
    train_folds = folds[folds['fold'] != fold].reset_index(drop=True)
    valid_folds = folds[folds['fold'] == fold].reset_index(drop=True)
    # batched and sorted for GPU limitation
    train_folds = sort_df_batch_lens(train_folds,CFG.batch_size)
    valid_folds = sort_df_batch_lens(valid_folds,CFG.batch_size)
    valid_labels = valid_folds['score'].values
    
    train_dataset = TrainDataset(CFG, train_folds)
    valid_dataset = TrainDataset(CFG, valid_folds)

    train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

    # ====================================================
    # model & optimizer
    # ====================================================
    model = CustomModel(CFG)
    #torch.save(model.config, OUTPUT_DIR+'config.pth')
    model.to(device)
    
    def get_optimizer_params(model, encoder_lr, decoder_lr, weight_decay=0.0):
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        freeze = ['embeddings.word_embeddings.weight','embeddings.LayerNorm.weight','embeddings.LayerNorm.bias']
        optimizer_parameters = [
           {'params': [p for n, p in model.model.named_parameters() if not any(nd in n for nd in no_decay) ], 
             'lr': encoder_lr, 'weight_decay': weight_decay},
            {'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': 0.0},
            {'params': [p for n, p in model.named_parameters() if "model" not in n],
             'lr': decoder_lr, 'weight_decay': 0.0}
        ]
        return optimizer_parameters

    optimizer_parameters = get_optimizer_params(model,
                                                encoder_lr=CFG.encoder_lr, 
                                                decoder_lr=CFG.decoder_lr,
                                                weight_decay=CFG.weight_decay)
    optimizer = AdamW(optimizer_parameters, lr=CFG.encoder_lr, eps=CFG.eps, betas=CFG.betas)
    
    # ====================================================
    # scheduler
    # ====================================================
    def get_scheduler(cfg, optimizer, num_train_steps):
        if cfg.scheduler == 'linear':
            scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps
            )
        elif cfg.scheduler == 'cosine':
            scheduler = get_cosine_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=cfg.num_cycles
            )
        return scheduler
    
    num_train_steps = int(len(train_folds) / CFG.batch_size * CFG.epochs)
    scheduler = get_scheduler(CFG, optimizer, num_train_steps)

    # ====================================================
    # loop
    # ====================================================
    criterion = pearsonLoss()
    #  nn.BCEWithLogitsLoss(reduction="mean")
    
    # ====================================
    # AWP
    # ===================================
    awp = AWP(model, optimizer, adv_lr=0.001, adv_eps=0.001) if CFG.awp else None
    
    
    best_score = 0.

    for epoch in range(CFG.epochs):

        start_time = time.time()
        
        # train
        avg_loss = train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device,awp)

        # eval
        avg_val_loss, predictions = valid_fn(valid_loader, model, criterion, device)
        
        # scoring
        score = get_score(valid_labels, predictions)

        elapsed = time.time() - start_time

        print(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        print(f'Epoch {epoch+1} - Score: <font color="red">{score:.4f}</font>')
        
        if best_score < score:
            best_score = score
            print(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(),
                        'predictions': predictions},
                        OUTPUT_DIR+f"{CFG.model_path.replace('/', '-')}_fold{fold}_best.pth")

    predictions = torch.load(OUTPUT_DIR+f"{CFG.model_path.replace('/', '-')}_fold{fold}_best.pth", 
                             map_location=torch.device('cpu'))['predictions']
    valid_folds['pred'] = predictions

    torch.cuda.empty_cache()
    gc.collect()
    
    return valid_folds

In [25]:
# test the parameters
model = CustomModel(CFG)

In [26]:
      #optimizer_parameters = [
       #    {'params': [p for n, p in model.model.named_parameters()  if ((not any(nd in n for nd in no_decay)) and (not any(nd in n for nd in freeze)))], 
        #     'lr': encoder_lr, 'weight_decay': weight_decay},
         #   {'params': [p for n, p in model.model.named_parameters() if ((any(nd in n for nd in no_decay)) and (not any(nd in n for nd in freeze)))],
          #   'lr': encoder_lr, 'weight_decay': 0.0},
           # {'params': [p for n, p in model.named_parameters() if "model" not in n],
            # 'lr': decoder_lr, 'weight_decay': 0.0}
       # ]

In [27]:
class AWP:
    def __init__(self, model, optimizer, *, adv_param='weight',
                 adv_lr=0.001, adv_eps=0.001):
        self.model = model
        self.optimizer = optimizer
        self.adv_param = adv_param
        self.adv_lr = adv_lr
        self.adv_eps = adv_eps
        self.backup = {}

    def perturb(self, input_ids, attention_mask, token_type_ids, y, criterion):
        """
        Perturb model parameters for AWP gradient
        Call before loss and loss.backward()
        """
        self._save()  # save model parameters
        self._attack_step()  # perturb weights

    def _attack_step(self):
        e = 1e-6
        for name, param in self.model.named_parameters():
            if param.requires_grad and param.grad is not None and self.adv_param in name:
                grad = self.optimizer.state[param]['exp_avg']
                norm_grad = torch.norm(grad)
                norm_data = torch.norm(param.detach())

                if norm_grad != 0 and not torch.isnan(norm_grad):
                    # Set lower and upper limit in change
                    limit_eps = self.adv_eps * param.detach().abs()
                    param_min = param.data - limit_eps
                    param_max = param.data + limit_eps

                    # Perturb along gradient
                    # w += (adv_lr * |w| / |grad|) * grad
                    param.data.add_(grad, alpha=(self.adv_lr * (norm_data + e) / (norm_grad + e)))

                    # Apply the limit to the change
                    param.data.clamp_(param_min, param_max)

    def _save(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad and param.grad is not None and self.adv_param in name:
                if name not in self.backup:
                    self.backup[name] = param.clone().detach()
                else:
                    self.backup[name].copy_(param.data)

    def restore(self):
        """
        Restore model parameter to correct position; AWP do not perturbe weights, it perturb gradients
        Call after loss.backward(), before optimizer.step()
        """
        for name, param in self.model.named_parameters():
            if name in self.backup:
                param.data.copy_(self.backup[name])

### best batch size ?
we have two factores, 1 - the GPU limitation ( more important now, we want to build more complix model ), 2 - the time ( now is not the most important factor ) 
- we can work with the accumulation steps or not 

In [28]:
x = sort_df_batch_lens(train,64)
x['losses'] = x['batch_max_length'] - x['input_lengths']
total_loss = x.groupby(['batch_max_length'])['losses'].sum()
print(sum(total_loss))

13971


In [29]:
x = sort_df_batch_lens(train,32)
x['losses'] = x['batch_max_length'] - x['input_lengths']
total_loss = x.groupby(['batch_max_length'])['losses'].sum()
print(sum(total_loss))

7304


In [30]:
x = sort_df_batch_lens(train,16)
x['losses'] = x['batch_max_length'] - x['input_lengths']
total_loss = x.groupby(['batch_max_length'])['losses'].sum()
print(sum(total_loss))

3126


In [31]:
x = sort_df_batch_lens(train,8)
x['losses'] = x['batch_max_length'] - x['input_lengths']
total_loss = x.groupby(['batch_max_length'])['losses'].sum()
print(sum(total_loss))

1419


In [32]:
x = sort_df_batch_lens(train,4)
x['losses'] = x['batch_max_length'] - x['input_lengths']
total_loss = x.groupby(['batch_max_length'])['losses'].sum()
print(sum(total_loss))

555


the values not very accurate, the reason is that the data will be splitited to train and validate sets, but very near

In [33]:
x.tail(5)

,id,anchor,target,context,score,score_map,fold,context_text,targets,targets_list,n_targets,inputs,input_lengths,batch_max_length,losses
19678,ac2f04053fb8376f,mayenite,inorganic,H01,0.5,2,4,ELECTRICITY. BASIC ELECTRIC ELEMENTS,carbon,[carbon],2,mayenite[SEP]inorganic[SEP]ELECTRICITY. BASIC ...,16,16,0
14345,185ca3b9aebc40ab,gas leak,gas leak detection,G08,0.5,2,1,PHYSICS. SIGNALLING,,[],1,gas leak[SEP]gas leak detection[SEP]PHYSICS. S...,15,16,1
47,09db0a872aa1d88c,abatement,noise abatement,H01,0.5,2,0,ELECTRICITY. BASIC ELECTRIC ELEMENTS,,[],1,abatement[SEP]noise abatement[SEP]ELECTRICITY....,15,16,1
18242,bea5853b45aa2e21,lifting finger,spring,G04,0.5,2,2,PHYSICS. HOROLOGY,,[],1,lifting finger[SEP]spring[SEP]PHYSICS. HOROLOG...,13,16,3
5080,154b9aa33a6e90f8,cervical support,base,G02,0.5,2,4,PHYSICS. OPTICS,,[],1,cervical support[SEP]base[SEP]PHYSICS. OPTICS[...,13,13,0


In [34]:
del x

In [35]:
# training on validations
#for fold in (range(CFG.tr_n_fold)):
for fold in range(CFG.n_fold):
    print(f"-------------------------- {fold+1} --------------------------")
    train_loop(train,fold)

-------------------------- 1 --------------------------
Epoch: [1][0/1821] Elapsed 0m 2s (remain 72m 8s) Loss: 0.3503(0.3503) Grad: nan  LR: 0.00002000  
Epoch: [1][100/1821] Elapsed 0m 55s (remain 15m 53s) Loss: 0.2100(0.2357) Grad: 36715.8281  LR: 0.00002000  
Epoch: [1][200/1821] Elapsed 1m 34s (remain 12m 38s) Loss: 0.2842(0.2432) Grad: 78378.3828  LR: 0.00002000  
Epoch: [1][300/1821] Elapsed 2m 8s (remain 10m 48s) Loss: 0.2822(0.2447) Grad: 34909.4922  LR: 0.00002000  
Epoch: [1][400/1821] Elapsed 2m 40s (remain 9m 27s) Loss: 0.2837(0.2436) Grad: 49316.2148  LR: 0.00001999  
Epoch: [1][500/1821] Elapsed 3m 10s (remain 8m 21s) Loss: 0.0659(0.2343) Grad: 19110.5273  LR: 0.00001999  
Epoch: [1][600/1821] Elapsed 3m 39s (remain 7m 24s) Loss: 0.0986(0.2150) Grad: 19776.1562  LR: 0.00001999  
Epoch: [1][700/1821] Elapsed 4m 5s (remain 6m 32s) Loss: 0.0415(0.1988) Grad: 11126.5977  LR: 0.00001998  
Epoch: [1][800/1821] Elapsed 4m 30s (remain 5m 45s) Loss: 0.0803(0.1845) Grad: 31405.9590

> pearsonLoss function
> > Epoch 4 - fold 0 - Score: 0.7947 with smooth = 1 \
> > Epoch 3 - fold 0 -  <span style="color: red;">Score: 0.8134</span>  with smooth = 0 and sigmoid \
> > Epoch 4 - fold 0 -  0.8050  with smooth = 1 and softmax \
> > Epoch 5 - fold 0 -  <span style="color: red;"> Score: 0.8226 </span> with groupby and sigmoid

> other loss function and tricks  
> > Epoch 4 - fold 0 - Score: 0.8084 nn.BCEWithLogitsLoss(reduction="mean")

In [36]:
for model in os.scandir('/kaggle/working/models'):
    if model.is_file():
        print(model.name)

-kaggle-input-debertav3base_fold3_best.pth
-kaggle-input-debertav3base_fold0_best.pth
-kaggle-input-debertav3base_fold4_best.pth
-kaggle-input-debertav3base_fold2_best.pth
-kaggle-input-debertav3base_fold1_best.pth


### submission with simple ensemple method

In [37]:

# Test data set 
def prepare_text(text):
    inputs = CFG.tokenizer(text,
                           add_special_tokens=True,
                           max_length=CFG.max_len,
                           padding="max_length",
                          # return_token_type_ids = False,
                          # return_attention_mask = False,
                           return_offsets_mapping=False) # pass only text
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs   

class TestDataset(torch.utils.data.Dataset):
    def __init__(self,df):
        # super(TrainDataset, self).__init__(self)
        self.size = df.shape[0]
        self.input = df.inputs.values

    def __len__(self):
        return self.size
    
    def __getitem__(self, item):
        inputs = prepare_text(self.input[item])
        return inputs

In [38]:
# load the test set in dataloader and predict it with a loaded model 
def predict(model, test):
    '''
    load the test set in dataloader and predict it with a loaded model 
    '''
    
    # prepare the input
    test['inputs'] = test['anchor'] + '[SEP]' + test['target'] + '[SEP]'  + test['context_text'] + '[SEP]' + test['targets']
    
    # tok_inputs = (test['inputs'].apply(lambda x : prepare_text(CFG,x))).values
    inputs = TestDataset(test)
    test_loader = DataLoader(inputs,
                              batch_size=test.shape[0],
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
    
    for inputs in test_loader:
            with torch.no_grad():
                preds = model(inputs)

   # preds = preds.astype(float)
    #preds = model.predict(tok_inputs).predictions.astype(float)
    preds = np.clip(preds, 0, 1) # clip the values, the target values 
    preds = preds.flatten()
    results =  pd.DataFrame({
                'id': test['id'],
                'score': preds
                            })
    return results


#torch.load(OUTPUT_DIR+f"{CFG.model_path.replace('/', '-')}_fold{fold}_best.pth", 
#                             map_location=torch.device('cpu'))['predictions']


def  predict_ovar_all(test):
    '''
    predict ovar all avaliables models and combine the results
    
    '''
    results =  pd.DataFrame()
    results['id'] = test['id']
    
    i = 0
    for obj in os.scandir('/kaggle/working/models'):
        if obj.is_file():
            print(f"model: {obj.name}")
            
            model = CustomModel(CFG)
            checkpoint = torch.load(obj.path)['model']
            model.load_state_dict(checkpoint)
         #   print(model)
            #model.eval()
            if i > 2:
                break
            
            result = predict(model, test)
            results[f"{obj.name}"] = result['score']
            i = i + 1
    return results
        
    
    

In [39]:
results = predict_ovar_all(test)
results.head()

model: -kaggle-input-debertav3base_fold3_best.pth
model: -kaggle-input-debertav3base_fold0_best.pth
model: -kaggle-input-debertav3base_fold4_best.pth
model: -kaggle-input-debertav3base_fold2_best.pth


,id,-kaggle-input-debertav3base_fold3_best.pth,-kaggle-input-debertav3base_fold0_best.pth,-kaggle-input-debertav3base_fold4_best.pth
0,4112d61851461f60,0.541236,0.363553,0.333909
1,09e418c93a776564,1.000000,0.896179,1.000000
2,36baf228038e314b,0.226740,0.264140,0.608859
3,1f37ead645e7f0c8,0.000000,0.072868,0.000000
4,71a5b6ad068d531f,0.000000,0.000000,0.000000


In [40]:
results

,id,-kaggle-input-debertav3base_fold3_best.pth,-kaggle-input-debertav3base_fold0_best.pth,-kaggle-input-debertav3base_fold4_best.pth
0,4112d61851461f60,0.541236,0.363553,0.333909
1,09e418c93a776564,1.000000,0.896179,1.000000
2,36baf228038e314b,0.226740,0.264140,0.608859
3,1f37ead645e7f0c8,0.000000,0.072868,0.000000
4,71a5b6ad068d531f,0.000000,0.000000,0.000000
5,474c874d0c07bd21,0.248652,0.481068,0.390612
6,442c114ed5c4e3c9,0.000000,0.039494,0.000000
7,b8ae62ea5e1d8bdb,0.000000,0.000000,0.000000
8,faaddaf8fcba8a3f,0.000000,0.000000,0.000000
9,ae0262c02566d2ce,1.000000,1.000000,1.000000


In [41]:
numeric_columns = results.select_dtypes(include='number').columns
results['mean'] = results[numeric_columns].mean(axis=1)
#results.groupby(['id']).mean()
results.head()

,id,-kaggle-input-debertav3base_fold3_best.pth,-kaggle-input-debertav3base_fold0_best.pth,-kaggle-input-debertav3base_fold4_best.pth,mean
0,4112d61851461f60,0.541236,0.363553,0.333909,0.412900
1,09e418c93a776564,1.000000,0.896179,1.000000,0.965393
2,36baf228038e314b,0.226740,0.264140,0.608859,0.366580
3,1f37ead645e7f0c8,0.000000,0.072868,0.000000,0.024289
4,71a5b6ad068d531f,0.000000,0.000000,0.000000,0.000000


## to the competition 

In [42]:
submission = pd.DataFrame({
    'id': results['id'],
    'score': results['mean']
})

submission.to_csv('submission.csv', index=False)

In [43]:
submission

,id,score
0,4112d61851461f60,0.412900
1,09e418c93a776564,0.965393
2,36baf228038e314b,0.366580
3,1f37ead645e7f0c8,0.024289
4,71a5b6ad068d531f,0.000000
5,474c874d0c07bd21,0.373444
6,442c114ed5c4e3c9,0.013165
7,b8ae62ea5e1d8bdb,0.000000
8,faaddaf8fcba8a3f,0.000000
9,ae0262c02566d2ce,1.000000
